In [ ]:
!pip install openai flask flask-ngrok2 transformers
!python3 -m pip install -U git+https://github.com/facebookresearch/audiocraft#egg=audiocraft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 783.0 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 21.8 MB/s eta 0:00:00
  Cloning https://github.com/facebookresearch/audiocraft to /tmp/pip-install-mrunb6sd/audiocraft_d8a7a1c8570b498ca7945de5758b1d65
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/audiocraft /tmp/pip-install-mrunb6sd/audiocraft_d8a7a1c8570b498ca7945de5758b1d65
  Resolved https://github.com/facebookresearch/audiocraft to commit c09cd0cb9075e15fc96e7c791d4ba6878079601c
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.0/31.0 MB 47.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 4.1 MB/s 

In [ ]:
import openai

In [ ]:
openai.api_key = '[redacted]'

In [ ]:
!ngrok authtoken [redacted]

In [ ]:
import os
import threading
from flask import Flask, request
from flask_ngrok2 import run_with_ngrok
import random
from transformers import pipeline
from audiocraft.models import musicgen
from audiocraft.utils.notebook import display_audio
import torch
from audiocraft.data.audio import audio_write
from flask import send_file

app = Flask(__name__)
run_with_ngrok(app=app, auth_token='[redacted]')

@app.route('/')
def hello():
    return "Hello, Flask on Google Colab!"

@app.route('/predict')
def predict():
    description = request.args.get('prompt')
    response = openai.Image.create(
    prompt=description,
        n=1,
        size="1024x1024"
    )
    image_url = response['data'][0]['url']
    return {'url': image_url}

@app.route('/sentiment')
def sentiment():
    sent_pipeline = pipeline("text-classification")
    dt = sent_pipeline(request.args.get('text'))
    if dt['label'] == 'POSITIVE':
        pos_percentage = dt['score']
        neg_percentage = 1 - dt['score']
    else:
        neg_percentage = dt['score']
        pos_percentage = 1 - dt['score']

    blue = random.randint(0, 80)
    red = neg_percentage * 254
    green = pos_percentage * 254
    return {'label': dt['label'], 'score': dt['score'], 'color': f"#{int(red):02x}{int(green):02x}{int(blue):02x}"}

@app.route('/music')
def music():
    model = musicgen.MusicGen.get_pretrained('medium', device='cpu')
    model.set_generation_params(duration=request.args.get('duration'))
    res = model.generate([request.args.get('prompt')], progress=True)
    audio_write('test', res.cpu()[0][0], model.sample_rate, strategy="loudness", loudness_compressor=True)
    return send_file('test.wav')

if __name__ == '__main__':
    app.run()